# 네이버 영화평 수집


In [1]:
import requests

In [2]:
import lxml.html

In [24]:
# 리뷰 모음
reviews = []

# 네이버 영화평 주소
url = 'http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121051&type=after&onlyActualPointYn=Y&page={}'

for page in range(100, 500):
    res = requests.get(url.format(page))
    element = lxml.html.fromstring(res.text)
    for e in element.xpath('.//div[@class="score_result"]//li'):
        star = e.find('.//div[@class="star_score"]//em').text_content()
        comment = e.find('.//div[@class="score_reple"]//p').text_content()
        reviews.append([star, comment[3:]])

In [25]:
len(reviews)

2495

## csv 파일로 저장

In [26]:
import csv

In [27]:
with open('naver_review.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow(['star', 'comment'])
    w.writerows(reviews)

# Term-Document Matrix

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from konlpy.tag import Twitter

In [5]:
tagger = Twitter()

In [10]:
def get_word(text):
    nouns = tagger.nouns(text)
    return [noun for noun in nouns if len(noun) > 1]

In [11]:
cv = CountVectorizer(tokenizer=get_word, max_features=1000)

In [12]:
tdm = cv.fit_transform([r[1] for r in reviews])

In [14]:
noun_list = cv.get_feature_names()

## 저장

In [15]:
import numpy

In [16]:
numpy.save('tdm.npy', tdm)

In [ ]:
with open('nouns.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(noun_list))